In [1]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
import shutil
import math
import inspect

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

**Load the initial layers**

In [2]:
arcpy.ResetEnvironments()
gdb_name = "Land_Use.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

In [3]:
crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')

in_data = ["SBC_Parcels", "SBC_Ag", "SBC_Lakes", "SBC_Desert", "SBC_Golf", "SBC_Rx_Grz"]

for sf in in_data:
    path = os.path.join(THIS_FOLDER, "in_data/{}.shp".format(sf))
    arcpy.CopyFeatures_management(path, sf)
    arcpy.Project_management(sf, "{}_rpj".format(sf), crs)

**We are only interested in large lakes (>10 acres)**

In [4]:
selection = arcpy.management.SelectLayerByAttribute("SBC_Lakes_rpj", "NEW_SELECTION", "Shape_Area > 435600", None)
arcpy.CopyFeatures_management(selection, "Lg_Lakes")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Land_Use.gdb\\Lg_Lakes'>

**Now we merge the land-use categories together and set the fields accordingly**

In [5]:
lu_cats = ["SBC_Ag_rpj", "Lg_Lakes", "SBC_Desert_rpj", "SBC_Golf_rpj", "SBC_Rx_Grz_rpj"]

arcpy.Merge_management(lu_cats, "LU_merged")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Land_Use.gdb\\LU_merged'>

In [6]:
arcpy.AddField_management("LU_merged", "golf", "LONG")
arcpy.AddField_management("LU_merged", "ag", "LONG")
arcpy.AddField_management("LU_merged", "lake", "LONG")
arcpy.AddField_management("LU_merged", "desert", "LONG")
arcpy.AddField_management("LU_merged", "rx_grz", "LONG")

codeblock = """
def getClass(a):
    if a is None:
        return 0
    else:
        return 1
"""

arcpy.CalculateField_management("LU_merged",  "golf", "getClass(!apn!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("LU_merged",  "ag", "getClass(!CropCatego!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("LU_merged",  "lake", "getClass(!WETLAND_TY!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("LU_merged",  "desert", "getClass(!WHRTYPE!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("LU_merged",  "rx_grz", "getClass(!FolderPath!)", "PYTHON3", codeblock)


FCfields = [f.name for f in arcpy.ListFields("LU_merged")]
DontDeleteFields = ['Shape_Length', 'Shape_Area', 'OBJECTID', 'OBJECTID_1', 'Shape', "golf", "ag", "lake", "desert", "rx_grz"]
fields2Delete = list(set(FCfields) - set(DontDeleteFields))
arcpy.DeleteField_management("LU_merged", fields2Delete)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Land_Use.gdb\\LU_merged'>

**Create a single field to specify land-use category (1=ag, 2=desert, 3=lglake, 4=golf, 0=everything else)**

In [7]:
arcpy.AddField_management("LU_merged", "LU_Cat", "LONG")

expression = "getClass(!golf!, !ag!, !lake!, !desert!, !rx_grz!)"

codeblock = """
def getClass(g, a, l, d, r):
    if a == 1:
        return 1
    elif d == 1:
        return 2
    elif l == 1:
        return 3
    elif g == 1:
        return 4
    elif r == 1:
        return 5
    else:
        return 0
"""

arcpy.CalculateField_management("LU_merged",  "LU_Cat", expression, "PYTHON3", codeblock)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Land_Use.gdb\\LU_merged'>

**Export the land-use category to out_data**

In [8]:
dir_path = os.path.join(THIS_FOLDER, "out_data/lu")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

arcpy.conversion.FeatureClassToShapefile("LU_merged", 
                                        os.path.join(THIS_FOLDER, "out_data/lu"))

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\out_data\\lu'>